In [110]:
# Initial imports
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

%matplotlib inline

In [111]:
# Import Verlander dataset 

verlander_df = pd.read_csv(Path("../resources/verlander_update.csv"),
                            index_col='game_date',
                            parse_dates=True,
                            infer_datetime_format=True)

display(verlander_df.head())

,pitch_type,player_name,batter,pitcher,events,description,zone,des,stand,p_throws,...,on_1b,outs_when_up,inning,at_bat_number,pitch_number,pitch_name,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment
game_date,,,,,,,,,,,,,,,,,,,,,
2022-09-16,SL,"Verlander, Justin",669127,434378,strikeout,swinging_strike,9,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,5,Slider,0,2,Infield shift,Standard
2022-09-16,FF,"Verlander, Justin",669127,434378,NaN,foul,3,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,4,4-Seam Fastball,0,2,Infield shift,Standard
2022-09-16,SL,"Verlander, Justin",669127,434378,NaN,called_strike,13,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,3,Slider,0,2,Infield shift,Standard
2022-09-16,FF,"Verlander, Justin",669127,434378,NaN,foul,1,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,2,4-Seam Fastball,0,2,Infield shift,Standard
2022-09-16,SL,"Verlander, Justin",669127,434378,NaN,ball,8,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,1,Slider,0,2,Infield shift,Standard


In [112]:
# Clean dataset 

verlander_df = verlander_df.drop(columns=['des', 
'pitch_name', 
'at_bat_number', 
'inning', 
'zone', 
'type', 
'player_name', 
'batter', 
'pitcher', 
'events',
'bb_type',
'hit_location'])

display(verlander_df.head())

,pitch_type,description,stand,p_throws,home_team,away_team,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment
game_date,,,,,,,,,,,,,,,,,
2022-09-16,SL,swinging_strike,R,R,HOU,OAK,1,2,NaN,NaN,NaN,2,5,0,2,Infield shift,Standard
2022-09-16,FF,foul,R,R,HOU,OAK,1,2,NaN,NaN,NaN,2,4,0,2,Infield shift,Standard
2022-09-16,SL,called_strike,R,R,HOU,OAK,1,1,NaN,NaN,NaN,2,3,0,2,Infield shift,Standard
2022-09-16,FF,foul,R,R,HOU,OAK,1,0,NaN,NaN,NaN,2,2,0,2,Infield shift,Standard
2022-09-16,SL,ball,R,R,HOU,OAK,0,0,NaN,NaN,NaN,2,1,0,2,Infield shift,Standard


In [124]:
# Encode 1st, 2nd and 3rd bases with 1s and 0s

onbase_df = verlander_df[['on_1b', 'on_2b', 'on_3b']]

onbase_df.loc[onbase_df['on_1b']==np.nan] = 0
onbase_df.loc[onbase_df['on_2b']==np.nan] = 0
onbase_df.loc[onbase_df['on_3b']==np.nan] = 0

onbase_df.loc[onbase_df['on_1b']>0] = 1
onbase_df.loc[onbase_df['on_2b']>0] = 1
onbase_df.loc[onbase_df['on_3b']>0] = 1

# for label, content in onbase_df.iterrows():
#     label.replace(np.nan, 0)
# print("phew finally finished")

#onbase_df['on_1b'].value_counts()
display(onbase_df[1220:1400])

/var/folders/zx/3xfmqcd54f3fdz2hfpnll0_80000gn/T/ipykernel_1529/3735095843.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onbase_df.loc[onbase_df['on_1b']==np.nan] = 0
/var/folders/zx/3xfmqcd54f3fdz2hfpnll0_80000gn/T/ipykernel_1529/3735095843.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onbase_df.loc[onbase_df['on_2b']==np.nan] = 0
/var/folders/zx/3xfmqcd54f3fdz2hfpnll0_80000gn/T/ipykernel_1529/3735095843.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

,on_1b,on_2b,on_3b
game_date,,,
2022-06-12,1.0,1.0,1.0
2022-06-12,1.0,1.0,1.0
2022-06-12,1.0,1.0,1.0
2022-06-12,1.0,1.0,1.0
2022-06-12,1.0,1.0,1.0
...,...,...,...
2022-06-07,NaN,NaN,NaN
2022-06-07,NaN,NaN,NaN
2022-06-07,NaN,NaN,NaN


In [114]:
# Split into X and y 

X = verlander_df.drop(columns='pitch_type')
y= verlander_df['pitch_type']

In [115]:
# Use get_dummies to encode categorical variables 

X = pd.get_dummies(X)


In [116]:
# Initialize the Deep Learning Neural Network model

nn_v0 = Sequential()

In [117]:
# Design the network architecture 

# Define the model - deep neural net
number_input_features = len(verlander_df.columns)
hidden_nodes_layer1 = number_input_features/2
hidden_nodes_layer2 = hidden_nodes_layer1/2
hidden_nodes_layer3 = hidden_nodes_layer2/2

# Add first layer
nn_v0.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Add second layer 
nn_v0.add(Dense(units=hidden_nodes_layer2, activation='relu'))

# Add third layer 
nn_v0.add(Dense(units=hidden_nodes_layer3, activation='relu'))